In [1]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from gp_pref_elicit_luisa import dataset as data
from gp_pref_elicit_luisa import gaussian_process as GP
from momabs_bayesian import bayes_logistic as bayes_logistic
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users
from logistic_user import LogisticDecisionMaker as logistic 

In [2]:
# initializing dataset class
utils_comparisons = data.DatasetPairwise(num_objectives=2)
GP = GP.GPPairwise(num_objectives=2)

In [3]:
# synthetic pareto coverage set
synthetic_pcs = np.array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])

In [4]:
# generating random points from synthetic pcs 
start_points = synthetic_pcs[np.random.choice(synthetic_pcs.shape[0], size=2, replace=False)]
start_points

array([[0.18584726, 0.98334638],
       [0.33305315, 0.94955291]])

In [5]:
 # making an object of LogisticDecisionMaker class
user_simulation = logistic(no_obj=2, num_features=4)
# comparing the initial points for getting the current best
current_best = user_simulation.exact_compare(start_points[0], start_points[1]) 
current_best

array([0.18584726, 0.98334638])

In [6]:
# making an object of LogisticDecisionMaker class
model_space = logistic(no_obj=2, num_features=4) 
# getting the model space which contains samples from a multivariate gaussian dsitribution
model_space = model_space.sample_model()
model_space

array([-1.19157865,  0.24124023, -1.22464401,  0.46851578])

In [7]:
# getting points according to thompson sampling 
thompson_point = user_simulation.thompson_sampled_point(synthetic_pcs)
thompson_point

array([0., 1.])

In [10]:
# comparing the current best and the thompson sampled point
user_simulation.exact_compare(current_best, thompson_point)

array([0.18584726, 0.98334638])

In [11]:
# passing the comparison to bayesian logistic regression to get the mean and the covariance matrix 
user_simulation.current_map()

(array([0.25144507, 0.25102431, 0.25102431, 0.25144507]),
 array([9.19676863, 9.18309093, 9.18309093, 9.19676863]))

As seen above, the covariance matrix has independent variances.

In [ ]:
# TODO: loop for BLR with stopping condition: if the current best point is already in the list of excluded points
# terminate the loop else continue